Text Classification - Sentiment Analysis Model
- https://huggingface.co/lxyuan/distilbert-base-multilingual-cased-sentiments-student
- https://huggingface.co/jarvisx17/japanese-sentiment-analysis

Zero-Shot Classification
- https://huggingface.co/MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7

In [76]:
import pandas as pd
from tqdm import tqdm

from transformers import pipeline

In [ ]:
!ls ../tmp

In [3]:
df_train = pd.read_csv('../tmp/hatespeech_train.csv')
df_test = pd.read_csv('../tmp/hatespeech_test.csv')
df_train.shape, df_test.shape

((4656, 5), (600, 4))

In [4]:
df_train.head()

,id,source,text,label,hold
0,b1b0d5e6c,newsplus,手引きしたのは高い確率で大院君だよなぁ,0,2
1,a5e29c5f1,livejupiter,いや別にワイが困るわけじゃないからええけど対応大変やなぁと思ってな,0,1
2,bd72cb57e,newsplus,そこで家族会すら総スカンを食らった極左を出すあたり頭が逝ってるな\n結局動かぬ証拠だから動か...,0,4
3,66387bae8,news4vip,もともとB'zが好きだったんだが松本のソロアルバムでtake5カバーしててな\nそれで原曲聴...,0,1
4,b80dc3a94,livejupiter,スタンドって本体の内面現すんだよな\n\nいろいろ想像するとおもろいわ\n何でDIOは時止め...,0,2


In [5]:
df_test.head()

,id,source,text,label
0,e3d3e578b,news4vip,まぁ、俺の言ってることは余りあてにしないでくれwwww\n\n必ずいい人は現れるよ,0
1,a728f26db,news4vip,すまないがそれはレンジのほうか？\nそれともトースター？,0
2,c53e66dc9,livejupiter,そら大不況で独り勝ちした金持ちを殺してしかも自分もちゃんと腹切ってるからな\n今で言うならリ...,0
3,fb3b600fb,newsplus,外国や外国人に税金や公的資金を使うのは違法と言ってくれたら、民進党の支持率は爆上げだろうな。,0
4,ef9cd1326,newsplus,中国人は、日本の年金制度にものらないとw,0


In [6]:
df_train[df_train['label']==1]['text']

18                      女なんてまともなやつはいないぞ\n彼女なんて絶対に作るもんじゃない
30                            ちなみにチンパンジーは99％な、韓国人は猿以下だってよ
31      人に大怪我させた飼い犬や狂犬病のおそれがある野良犬は刹処分に決まってんだろ\n犬の話を人間に...
35      むしろ日本が勝ってるところ見つけるのが難しいからな\n自動車とカメラぐらいや\nこんなお先真...
39      相変わらず韓国はキチガイだなあ\n報道機関が「勝手な仇名」で他国の君主を呼んでる\n例えば日...
                              ...                        
4526    女優みたいな清純というか普通の落ち着いた女性が全然いないのが悲しい\nアメリカ被れの女なんか...
4570                                      女は論理的思考が苦手な人が多い
4572    男は皆のためを考えるが、女は女のためしか考えない。\nこれ事実な。\nその証拠に女の政治家と...
4605                         何語かも解からんわ\n死ねよウンコ在日アイヌめ\n殺すぞ
4608                                イスラムは多文化共生の悪影響の最もたるもの
Name: text, Length: 271, dtype: object

In [7]:
df_train[df_train['label']==0]['text']

0                                     手引きしたのは高い確率で大院君だよなぁ
1                       いや別にワイが困るわけじゃないからええけど対応大変やなぁと思ってな
2       そこで家族会すら総スカンを食らった極左を出すあたり頭が逝ってるな\n結局動かぬ証拠だから動か...
3       もともとB'zが好きだったんだが松本のソロアルバムでtake5カバーしててな\nそれで原曲聴...
4       スタンドって本体の内面現すんだよな\n\nいろいろ想像するとおもろいわ\n何でDIOは時止め...
                              ...                        
4651                                         すまん。\nありがとう。
4652    人集まったり伸びたらそうするで\nあっちで１からやっても誰もおらんからな\nここでもおらんか...
4653    （；ﾟДﾟ）スマホだったら、おーぷんstormってのがありますね\n今スマホにインスコして書...
4654    実際には戦わなきゃ強姦されて略奪されて殺されるだけ\n日本もベトナムもイラクも米軍兵に蹂躙さ...
4655     女が近くにいるとムラムラして勉強出来ない\n家にいるとシコってしまって勉強出来ない\nどうすれば
Name: text, Length: 4385, dtype: object

In [13]:
# label=1（ヘイトスピーチ） examples
label1_1 = df_train[df_train['label']==1]['text'][31]
label1_2 = df_train[df_train['label']==1]['text'][39]
label1_3 = df_train[df_train['label']==1]['text'][4526]

# label=0（ヘイトスピーチでない） examples
label0_1 = df_train[df_train['label']==0]['text'][1]
label0_2 = df_train[df_train['label']==0]['text'][3]
label0_3 = df_train[df_train['label']==0]['text'][4652]

examples = [label1_1, label1_2, label1_3, label0_1, label0_2, label0_3]

## Text Classification - Sentiment Analysis Model

### lxyuan/distilbert-base-multilingual-cased-sentiments-student

In [10]:
distilbert_base_multilingual_cased_sentiments_student = pipeline(
    model="lxyuan/distilbert-base-multilingual-cased-sentiments-student", 
    return_all_scores=True
)
distilbert_base_multilingual_cased_sentiments_student

/opt/anaconda3/envs/PyTorch/lib/python3.8/site-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [11]:
distilbert_base_multilingual_cased_sentiments_student(label1_1)

[[{'label': 'positive', 'score': 0.17284806072711945},
  {'label': 'neutral', 'score': 0.16055375337600708},
  {'label': 'negative', 'score': 0.6665981411933899}]]

In [24]:
for tx in examples:
    print(tx)
    negative_score = distilbert_base_multilingual_cased_sentiments_student(tx)[0][2]['score']
    print('Negative score: ', negative_score)
    print('----------------------')

人に大怪我させた飼い犬や狂犬病のおそれがある野良犬は刹処分に決まってんだろ
犬の話を人間にすり替えるとか、お前ブサヨかよｗ
ちなみに朝鮮ジンは人とことなるから野生動物扱いでかまわんやろｗ
Negative score:  0.6665981411933899
----------------------
相変わらず韓国はキチガイだなあ
報道機関が「勝手な仇名」で他国の君主を呼んでる
例えば日本の新聞がエリザベス酋長とか言ったら無礼だろ？
韓国の大統領はもう酋長でもよさそうだけどさ
Negative score:  0.7662402391433716
----------------------
女優みたいな清純というか普通の落ち着いた女性が全然いないのが悲しい
アメリカ被れの女なんか日本にいる資格ないわ
Negative score:  0.7828424572944641
----------------------
いや別にワイが困るわけじゃないからええけど対応大変やなぁと思ってな
Negative score:  0.31211256980895996
----------------------
もともとB'zが好きだったんだが松本のソロアルバムでtake5カバーしててな
それで原曲聴いたらすげーかっこよかったから好きになってジャズにも興味持ったよ
BEATCRUSADERSはアニメでやってたBECKの影響だな
Negative score:  0.09889666736125946
----------------------
人集まったり伸びたらそうするで
あっちで１からやっても誰もおらんからな
ここでもおらんかったらこのまま終わりで
Negative score:  0.4421919286251068
----------------------


In [25]:
train_sentiment1 = distilbert_base_multilingual_cased_sentiments_student(df_train['text'].to_list())
test_sentiment1 = distilbert_base_multilingual_cased_sentiments_student(df_test['text'].to_list())
len(train_sentiment1), len(test_sentiment1)

(4656, 600)

In [36]:
train_sentiment1_negative = [neg[2]['score'] for neg in train_sentiment1]
test_sentiment1_negative = [neg[2]['score'] for neg in test_sentiment1]
len(train_sentiment1_negative), len(test_sentiment1_negative)

(4656, 600)

### jarvisx17/japanese-sentiment-analysis

In [37]:
japanese_sentiment_analysis = pipeline(
    model="jarvisx17/japanese-sentiment-analysis", 
    return_all_scores=True
)
japanese_sentiment_analysis

In [38]:
japanese_sentiment_analysis(label1_1)

[[{'label': 'negative', 'score': 0.9988510608673096},
  {'label': 'positive', 'score': 0.0011488927993923426}]]

In [39]:
japanese_sentiment_analysis(label0_1)

[[{'label': 'negative', 'score': 0.9111171364784241},
  {'label': 'positive', 'score': 0.08888282626867294}]]

In [43]:
for tx in examples:
    print(tx)
    negative_score = japanese_sentiment_analysis(tx)[0][0]['score']
    print('Negative score: ', negative_score)
    print('----------------------')

人に大怪我させた飼い犬や狂犬病のおそれがある野良犬は刹処分に決まってんだろ
犬の話を人間にすり替えるとか、お前ブサヨかよｗ
ちなみに朝鮮ジンは人とことなるから野生動物扱いでかまわんやろｗ
Negative score:  0.9988510608673096
----------------------
相変わらず韓国はキチガイだなあ
報道機関が「勝手な仇名」で他国の君主を呼んでる
例えば日本の新聞がエリザベス酋長とか言ったら無礼だろ？
韓国の大統領はもう酋長でもよさそうだけどさ
Negative score:  0.9997758269309998
----------------------
女優みたいな清純というか普通の落ち着いた女性が全然いないのが悲しい
アメリカ被れの女なんか日本にいる資格ないわ
Negative score:  0.9916503429412842
----------------------
いや別にワイが困るわけじゃないからええけど対応大変やなぁと思ってな
Negative score:  0.9111171364784241
----------------------
もともとB'zが好きだったんだが松本のソロアルバムでtake5カバーしててな
それで原曲聴いたらすげーかっこよかったから好きになってジャズにも興味持ったよ
BEATCRUSADERSはアニメでやってたBECKの影響だな
Negative score:  4.526908014668152e-05
----------------------
人集まったり伸びたらそうするで
あっちで１からやっても誰もおらんからな
ここでもおらんかったらこのまま終わりで
Negative score:  0.033542145043611526
----------------------


In [46]:
train_sentiment2 = japanese_sentiment_analysis(df_train['text'].to_list())
test_sentiment2 = japanese_sentiment_analysis(df_test['text'].to_list())
len(train_sentiment2), len(test_sentiment2)

(4656, 600)

In [49]:
#train_sentiment2

In [51]:
train_sentiment2_negative = [neg[0]['score'] for neg in train_sentiment2]
test_sentiment2_negative = [neg[0]['score'] for neg in test_sentiment2]
len(train_sentiment2_negative), len(test_sentiment2_negative)

(4656, 600)

## Zero-Shot Classification

### MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7

In [52]:
mDeBERTa_v3_base_xnli_multilingual = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli")
mDeBERTa_v3_base_xnli_multilingual

In [66]:
# 以下をヘイトスピーチを表す単語として、Zeroshot予測を実施
candidate_labels = ["悪意", "嫌悪", "差別"]

In [67]:
mDeBERTa_v3_base_xnli_multilingual(label1_1, candidate_labels, multi_label=True)

{'sequence': '人に大怪我させた飼い犬や狂犬病のおそれがある野良犬は刹処分に決まってんだろ\n犬の話を人間にすり替えるとか、お前ブサヨかよｗ\nちなみに朝鮮ジンは人とことなるから野生動物扱いでかまわんやろｗ',
 'labels': ['差別', '悪意', '嫌悪'],
 'scores': [0.9191533327102661, 0.8340898156166077, 0.267429918050766]}

In [68]:
mDeBERTa_v3_base_xnli_multilingual(label0_1, candidate_labels, multi_label=True)

{'sequence': 'いや別にワイが困るわけじゃないからええけど対応大変やなぁと思ってな',
 'labels': ['差別', '悪意', '嫌悪'],
 'scores': [0.342574805021286, 0.18324117362499237, 0.04959387332201004]}

In [69]:
for tx in examples:
    print(tx)
    zs_res = mDeBERTa_v3_base_xnli_multilingual(tx, candidate_labels, multi_label=True)
    print('Label: ', zs_res['labels'])
    print('Scores: ', zs_res['scores'])
    print({k:v for k,v in zip(zs_res['labels'], zs_res['scores'])})
    print('----------------------')

人に大怪我させた飼い犬や狂犬病のおそれがある野良犬は刹処分に決まってんだろ
犬の話を人間にすり替えるとか、お前ブサヨかよｗ
ちなみに朝鮮ジンは人とことなるから野生動物扱いでかまわんやろｗ
Label:  ['差別', '悪意', '嫌悪']
Scores:  [0.9191533327102661, 0.8340898156166077, 0.267429918050766]
{'差別': 0.9191533327102661, '悪意': 0.8340898156166077, '嫌悪': 0.267429918050766}
----------------------
相変わらず韓国はキチガイだなあ
報道機関が「勝手な仇名」で他国の君主を呼んでる
例えば日本の新聞がエリザベス酋長とか言ったら無礼だろ？
韓国の大統領はもう酋長でもよさそうだけどさ
Label:  ['悪意', '差別', '嫌悪']
Scores:  [0.9638715386390686, 0.9487082362174988, 0.844123125076294]
{'悪意': 0.9638715386390686, '差別': 0.9487082362174988, '嫌悪': 0.844123125076294}
----------------------
女優みたいな清純というか普通の落ち着いた女性が全然いないのが悲しい
アメリカ被れの女なんか日本にいる資格ないわ
Label:  ['嫌悪', '差別', '悪意']
Scores:  [0.9935705661773682, 0.9782067537307739, 0.9746213555335999]
{'嫌悪': 0.9935705661773682, '差別': 0.9782067537307739, '悪意': 0.9746213555335999}
----------------------
いや別にワイが困るわけじゃないからええけど対応大変やなぁと思ってな
Label:  ['差別', '悪意', '嫌悪']
Scores:  [0.342574805021286, 0.18324117362499237, 0.04959387332201004]
{'差別': 0.342574805021286, '悪意': 0.1

In [ ]:
train_zeroshot = []
#for tx in tqdm(examples):
for tx in tqdm(df_train['text']):
    zs_res = mDeBERTa_v3_base_xnli_multilingual(tx, candidate_labels, multi_label=True)
    train_zeroshot.append({k:v for k,v in zip(zs_res['labels'], zs_res['scores'])})
len(train_zeroshot)

 57%|█████▋    | 2641/4656 [2:07:52<29:09,  1.15it/s]     

In [ ]:
test_zeroshot = []
for tx in tqdm(df_test['text']):
    zs_res = mDeBERTa_v3_base_xnli_multilingual(tx, candidate_labels, multi_label=True)
    test_zeroshot.append({k:v for k,v in zip(zs_res['labels'], zs_res['scores'])})
len(test_zeroshot)

In [ ]:
train_zeroshot

In [ ]:
test_zeroshot

In [ ]:
train_malice　= [d['悪意'] for d in train_zeroshot]
train_hate　= [d['嫌悪'] for d in train_zeroshot]
train_discrimination　= [d['差別'] for d in train_zeroshot]
len(train_malice), len(train_hate), len(train_discrimination)

In [ ]:
test_malice　= [d['悪意'] for d in test_zeroshot]
test_hate　= [d['嫌悪'] for d in test_zeroshot]
test_discrimination　= [d['差別'] for d in test_zeroshot]
len(test_malice), len(test_hate), len(test_discrimination)

***

In [ ]:
df_train

In [ ]:
df_train['negative1'] = train_sentiment1_negative
df_train['negative2'] = train_sentiment2_negative
df_train['malice'] = train_malice
df_train['hate'] = train_hate
df_train['discrimination'] = train_discrimination

In [ ]:
df_train

In [ ]:
df_test

In [ ]:
df_test['negative1'] = test_sentiment1_negative
df_test['negative2'] = test_sentiment2_negative
df_test['malice'] = test_malice
df_test['hate'] = test_hate
df_test['discrimination'] = test_discrimination

In [ ]:
df_test

In [ ]:
!ls 

In [ ]:
df_train.to_csv('tmp/hatespeech_train_zeroshots.csv', index=False)
df_test.to_csv('tmp/hatespeech_test_zeroshots.csv', index=False)